## Ahmed Emam
## Imm.Num 64821

### Definition of task:
Analysis and prediction of air pollution in Madrid, Spain.
1. Analyse the multivariate dataset:
a. Are there significant correlations between the pollutant parameters?
b. The air quality can be described as the sum of all pollutants. Which parameters
have the biggest influence?
2. Generate a time series with average monthly values for Madrid, which describes the air
quality as the sum of all pollutants (9)!
a. In which months is the pollutant load greatest?
b. Perform a comprehensive analysis of the generated time series!
c. Choose a suitable stochastic model to model the time series and justify your
selection!
d. Perform a prediction of the time series (including 95% confidence interval) for
the following 2 years (until 04/2020) based on y

## Task 1:


In [ ]:
%matplotlib  inline
! pip install missingno
! pip install fbprophet
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import glob
import missingno as msno
# from fbprophet import Prophet

from datetime import datetime, timedelta
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import statsmodels.api as sm
from itertools import product
from math import sqrt
from sklearn.metrics import mean_squared_error 

from collections import defaultdict
from scipy.stats import boxcox
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.filters.hp_filter import hpfilter
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import os


In [ ]:

sns.set(rc={"figure.figsize": (20,10), "axes.titlesize" : 18, "axes.labelsize" : 12, 
            "xtick.labelsize" : 14, "ytick.labelsize" : 14 })

# Read in data

In [ ]:


frame = pd.read_csv('../input/madriddddd/madrid_mean_1h_raw.csv')

frame.head(10)

# Missing data?

In [ ]:
msno.matrix(frame);

In [ ]:
msno.bar(frame);

## statistical analysis summary for the data frame 


In [ ]:
measures= frame
measures.describe().round(decimals=3)

### Most dominant pollutants are NO_2 and O_3

 
 if we gave a look at the mean values of each of the pollutants, we can conclude that the most dominant polutants are No2 and O3 respectively, given that our assumptions that we the summation of the pollutants content is our goal. so the pollutant with the highest mean will be the dominant

_____________

### checking the correlations between different features (pollutants)

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(measures.corr(), square=True, annot=True, cmap='rainbow')

### we can see the highest correlation between BEN, CO, EBE and TOL, and that O_3 is the only with negative correlation value

 Positive Correlation: means that if feature A increases then feature B also increases or if feature A decreases then feature B also decreases. Both features move in tandem and they have a linear relationship.
 Negative Correlation: means that if feature A increases then feature B decreases and vice versa.
 No Correlation: No relationship between those two attributes.

 #### Correlation Matrix explaination:
 Each of those correlation types can exist in a spectrum represented by values from 0 to 1 where slightly or highly positive correlation features can be something like 0.5 or 0.7. If there is a strong and perfect positive correlation, then the result is represented by a correlation score value of 0.9 or 1.
 If there is a strong negative correlation, it will be represented by a value of -1.
 If your dataset has perfectly positive or negative attributes then there is a high chance that the performance of the model will be impacted by a problem called — “Multicollinearity”. Multicollinearity happens when one predictor variable in a multiple regression model can be linearly predicted from the others with a high degree of accuracy. This can lead to skewed or misleading results. Luckily, decision trees and boosted trees algorithms are immune to multicollinearity by nature. When they decide to split, the tree will choose only one of the perfectly correlated features. However, other algorithms like Logistic Regression or Linear Regression are not immune to that problem and you should fix it before training the model.
 
 ___________________________

## Task 2 


creating a sum column which will contain the summation of all pollutatnts values 

In [ ]:
measures['sum'] = measures.sum(axis = 1, skipna = True) 


we need to set the date as the index for the data frame


In [ ]:
date =pd.to_datetime(measures['date'])
date_index =pd.DatetimeIndex(date.values)
measures = measures.set_index(date_index)
measures.drop('date',axis=1,inplace =True)

In [ ]:
measures.head()

### Generate a time series with average monthly values for Madrid

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
Sum = measures['sum']

Sum /= Sum.max(axis=0)

(Sum.interpolate(method='time')
           .rolling(window=24*30).mean()
           .plot(ax=ax))

In [ ]:
from matplotlib.dates import MonthLocator, DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')
Sum_avg_month = Sum.resample('M').mean()
fig, ax = plt.subplots()
ax.plot(Sum_avg_month)

# format the ticks
ax.xaxis.set_major_locator(years)
#ax.xaxis.set_major_locator(months)
#ax.xaxis.set_major_formatter(DateFormatter('%m'))





# format the coords message box


ax.grid(True)

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()
plt.xticks(rotation='vertical')
plt.show()












In [ ]:
sorted_sum = measures['sum'].resample('M').mean().sort_values(ascending = False)
print(sorted_sum)
sorted_sum_index = sorted_sum.index
string = [sorted_sum_index.strftime('%m')]
flat_list = [] 

for sublist in string: 

     for item in sublist: 

        flat_list.append(item)
flat_list = list(map(int, flat_list))
flat_list = pd.Series(flat_list)
flat_list.describe()



In [ ]:
candidates = measures

In [ ]:
candidates['month'] = pd.to_datetime(candidates.index).month
candidates['year'] = pd.to_datetime(candidates.index).year
sns.lineplot(x='month',y='sum',hue= 'year',data=candidates.query('year>2001'))

#### Months with the highest value of pollutants are July and June
from the previous 3 plots and statistical analysis of dates with the highest average values of pollutants we can see that we have a surge in the pollutants levels in mid year 


.......................................................................................................................................................

## now let's check the seasonality and decombose the spectrum of the time series

In [ ]:
plt.style.use('fivethirtyeight') 
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
df=measures

In [ ]:

monthly_df = df.resample('M').mean()
plt_monthly = monthly_df['sum']
plt_monthly.plot(figsize=(15, 10))
plt.title('Madrid Pollutants from 2001-2019', fontsize=25)
plt.legend(loc='upper left')
plt.show()

In [ ]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    print('Null Hypothesis: Unit Root Present')
    print('Test Statistic < Critical Value => Reject Null')
    print('P-Value =< Alpha(.05) => Reject Null\n')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput[f'Critical Value {key}'] = value
    print (dfoutput, '\n')

def kpss_test(timeseries, regression='c'):
    # Whether stationary around constant 'c' or trend 'ct
    print ('Results of KPSS Test:')
    print('Null Hypothesis: Data is Stationary/Trend Stationary')
    print('Test Statistic > Critical Value => Reject Null')
    print('P-Value =< Alpha(.05) => Reject Null\n')
    kpsstest = kpss(timeseries, regression=regression)
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output[f'Critical Value {key}'] = value
    print (kpss_output, '\n')

In [ ]:
!pip install statsmodels

#### Results of Dickey-Fuller Test:

In [ ]:

adf_test(plt_monthly)
kpss_test(plt_monthly)


In [ ]:
decomposition_df = pd.DataFrame(monthly_df['sum'])
seasonal_a = seasonal_decompose(decomposition_df, model='additive')
seasonal_m = seasonal_decompose(decomposition_df, model='multiplicative')
fig_1 = seasonal_a.plot()
fig_2 = seasonal_m.plot()
fig_1.suptitle('Additive Seasonal Decomposition', fontsize=25)
fig_1.set_figheight(10)
fig_1.set_figwidth(20)
fig_2.suptitle('Multiplicative Seasonal Decomposition', fontsize=25)
fig_2.set_figheight(10)
fig_2.set_figwidth(20)
plt.show()

In [ ]:
import numpy as np
import statsmodels.api as sm

import statsmodels.formula.api as smf

In [ ]:
filter_df = pd.DataFrame(monthly_df['sum'])
sum_cycle, sum_trend =sm.tsa.filters.hpfilter(filter_df, lamb=129600)
filter_df['cycle'] = sum_cycle
filter_df['trend'] = sum_trend

filter_df.plot(figsize=(10, 5), title=' Pollutants Plot of Cycle and Trend')

### Forecasting by Prophet

In [ ]:
plt_monthly.head(5)

In [ ]:

monthly_df = df.resample('M').mean()
monthly_df['y'] = monthly_df['sum']
monthly_df['ds'] = monthly_df.index
monthly_df.tail(5)

In [ ]:
from fbprophet import Prophet
df = monthly_df
m = Prophet(seasonality_mode='multiplicative').fit(df)
future = m.make_future_dataframe(periods=356*2)
fcst = m.predict(future)









In [ ]:
len(fcst)
fcst.tail(3)

the forcasting ends in 12 may 2020 after 2 years

### plotting only the monthly data ******

In [ ]:
plt.plot(monthly_df['sum'],color = 'r', marker='o', label='actual monthly data')

### Plotting both real data and forcasting together

In [ ]:

m.plot(fcst,xlabel='time in year',uncertainty=False, plot_cap=False , ax = plt.axes())
plt.plot(monthly_df['sum'],color = 'r', marker='o', label='actual monthly data')
plt.legend()



plt.title('pollution forcast over madrid')
plt.ylabel('pollution level')





